In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, random_split

from torchvision import models
import torchvision.utils

import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
# helper function
def imshow(img, title):
    img = torchvision.utils.make_grid(img, normalize=True)
    npimg = img.numpy()
    fig = plt.figure(figsize=(5, 15))
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.title(title)
    plt.axis("off")
    plt.show()

In [3]:
# Use GPU if available, else use CPU
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Use metal for mac m1
# Check that MPS is available
if not torch.backends.mps.is_available():
    if not torch.backends.mps.is_built():
        print(
            "MPS not available because the current PyTorch install was not "
            "built with MPS enabled."
        )
    else:
        print(
            "MPS not available because the current MacOS version is not 12.3+ "
            "and/or you do not have an MPS-enabled device on this machine."
        )

else:
    device = torch.device("mps")
    print(device)

mps


<h1> Prepare Data</h1>

In [4]:
import dataset

# seed rng to reproduce
torch.manual_seed(42)
batch_size=128

dataset = dataset.create_dataset()
train_size = int(0.8 * len(dataset))  # 80% of data for training
test_size = len(dataset) - train_size  # Remaining 20% for testing

train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# print(len(dataset))
# print(len(train_loader))
# for batch_number, (inputs, labels) in enumerate(train_loader):
#     print(batch_number, inputs, labels)

100%|██████████| 4/4 [00:00<00:00, 635.50it/s]
/Users/yufangoh/Work/deepLearning_AlzheimerDetection/dataset.py:61: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace({"label": ad_labels.LABELS_MAP})


Use pretrained inception

In [5]:
model = models.inception_v3(pretrained=True)
model.aux_logits = False

for parameter in model.parameters():
    parameter.requires_grad = False

# model.fc = nn.Sequential(nn.Linear(model.fc.in_features, 10), nn.Linear(10, 2))
model.fc = nn.Sequential(nn.Linear(model.fc.in_features, 4))
# model.fc = nn.Sequential(nn.Linear(model.fc.in_features, 4), nn.Softmax(dim=1))

model = model.to(device)

loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(
    filter(lambda p: p.requires_grad, model.parameters()), lr=0.01
)

epoch_loss = 0.0
correct = 0
total = 0
num_epochs = 5

# iterate throug epochs
for epoch in range(num_epochs):
    model.train()
    total_batch = len(train_dataset) // batch_size

    for i, (batch_images, batch_labels) in enumerate(train_loader):
        images = batch_images.to(device)
        labels = batch_labels.to(device)

        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        # print("predicted", predicted.shape)
        # print(predicted)
        # print("batch_labels", batch_labels.shape)
        cost = loss(outputs, labels)

        correct += (predicted == labels).sum().item()
        total += labels.size(0)
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        if (i + 1) % 5 == 0:
            print(
                f"Epoch [{epoch + 1}/{num_epochs}], lter [{i+1}/{total_batch}] Loss: {cost.item()}, Cumulative Accuracy: {correct/total}"
            )

/Users/yufangoh/Work/deepLearning_AlzheimerDetection/venv/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/yufangoh/Work/deepLearning_AlzheimerDetection/venv/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch [1/5], lter [5/40] Loss: 2.643179178237915, Cumulative Accuracy: 0.3484375
Epoch [1/5], lter [10/40] Loss: 2.982750415802002, Cumulative Accuracy: 0.37578125
Epoch [1/5], lter [15/40] Loss: 1.6528375148773193, Cumulative Accuracy: 0.39895833333333336
Epoch [1/5], lter [20/40] Loss: 1.7381718158721924, Cumulative Accuracy: 0.420703125
Epoch [1/5], lter [25/40] Loss: 1.3135392665863037, Cumulative Accuracy: 0.431875
Epoch [1/5], lter [30/40] Loss: 0.959505558013916, Cumulative Accuracy: 0.44296875
Epoch [1/5], lter [35/40] Loss: 1.3966658115386963, Cumulative Accuracy: 0.45066964285714284
Epoch [1/5], lter [40/40] Loss: 1.016635775566101, Cumulative Accuracy: 0.4580078125
Epoch [2/5], lter [5/40] Loss: 0.8864609003067017, Cumulative Accuracy: 0.46857638888888886
Epoch [2/5], lter [10/40] Loss: 0.9367781281471252, Cumulative Accuracy: 0.475625
Epoch [2/5], lter [15/40] Loss: 0.9803450703620911, Cumulative Accuracy: 0.48480113636363636
Epoch [2/5], lter [20/40] Loss: 1.03998088836669

In [8]:
model.eval()

# model.load_state_dict(torch.load('inception_model.pth'))
epoch_loss = 0.0
correct = 0
total = 0

for images, labels in test_loader:
    images = images.to(device)
    labels = labels.to(device)

    with torch.no_grad():
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)

    total += labels.size(0)
    correct += (predicted == labels).sum().item()
    epoch_loss += loss(outputs, labels).item()
    accuracy = correct / total
    print(f"Accuracy of the model on the test images: {accuracy}")
# print("Accuracy of the model on the test images: %d %%" % accuracy)

Accuracy of the model on the test images: 0.5234375
Accuracy of the model on the test images: 0.5625
Accuracy of the model on the test images: 0.546875
Accuracy of the model on the test images: 0.5546875
Accuracy of the model on the test images: 0.5453125
Accuracy of the model on the test images: 0.5416666666666666
Accuracy of the model on the test images: 0.5513392857142857
Accuracy of the model on the test images: 0.55859375
Accuracy of the model on the test images: 0.5598958333333334
Accuracy of the model on the test images: 0.56484375
